# Lab 2
## Introduction
This lab introduces slope fields and a numerical differential equation solver. The solver is an improved version of Euler’s Method, which we will implement ourselves in future labs. Using these techniques involves a number of commands.

### Slope fields
Plot the slope field for the differential equation
\begin{align*}
\frac{\mathrm{d}y}{\mathrm{d}x} = x - y
\end{align*}
for $-1<x<5$ and $-2<y<4$.

This week, in addition to Seaborn, NumPy, and pandas, we will need Matplotlib and SciPy.

Matplotlib was the original popular Python plotting pakage. We need Matplotlib because Seaborn does not implement quiver plots. Fortunately, because Seaborn is built on top of Matplotlib, they play nicely together.

SciPy is NumPy's bigger sibling. We need SciPy to integrate the differential equations.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from numpy import meshgrid, linspace, sqrt
from numpy.testing import assert_almost_equal

from scipy.integrate import odeint

Now plot the slope field. A slope field is a special type of _quiver_ plot. We create NumPy arrays that say where to plot the line segments (`x` and `y`) and arrays to point them in the right direction (`1/L` and `S/L`).

Matplotlib is more hands-on that Seaborn, so you need extra steps like the `subplot` command to create the figure and axes in the first place, `set_title` to set the title of the plot, `plot.axis` command to set the aspect ratio of the plot, and various options within `quiver` to make it look good.

When we write `figsize=(5, 5)` in the inputs to `subplots`, are we creating a variable called `figsize`, or doing something else?

In [ ]:
x, y = meshgrid(linspace(-1, 5, 25), linspace(-2, 4, 25))
S = x - y
L = sqrt(1 + S**2)
fig, ax = plt.subplots(figsize=(5, 5))
q = ax.quiver(x, y, 1/L, S/L, scale=25, headwidth=0, headlength=0, color='blue')
ax.set_title('Slopefield for dy/dx = x - y')
plt.axis('equal');

`1/L` and `S/L` in the `create_quiver` command set the $x$ and $y$ lengths (components) of the line segment at each point in the grid.

Note that NumPy operates element-wise by default, so `x - y` creates an array of differences, and `S/L` creates an array of quotients. For `1/L`, NumPy does something special called _broadcasting_. It assumes that you meant "divide an array of ones by the elements of `L`".

The slope of the line segment is then $(S/L)/(1/L) = S$, and the length is
\begin{align*}
\sqrt{\left(\frac{1}{L}\right)^2 + \left(\frac{S}{L}\right)^2} &= \sqrt{\frac{1+S^2}{L^2}}\\
&= 1.
\end{align*}

### Numerical/graphical solution of an initial-value problem
Plot the (approximate) solution to the initial-value problem
\begin{align*}
\frac{\mathrm{d}y}{\mathrm{d}x} = x - y\qquad y(-1)=0
\end{align*}
for $-1 < x <5$. Find $y(5)$.

Here we use a numerical DE solver `scipy.integrate.odeint`, which we imported as `odeint`. To use `odeint`, we need to define the differential equation in a Python function and then feed it to `odeint`.

First define the function. Remember that in Python, [white space is important](https://xkcd.com/353/). That is, you have to indent the contents of your function or Python will complain. Most of the time your Jupyter Notebook will figure out your intentions and auto-indent.

In [ ]:
def diff_eq(y, x):
    return x - y

- The `def` keyword tells Python you would like to define a function.
- In this case the function is called `diff_eq` and takes arguments `y` and `x`.
- The `return` statement tells Python what you would like to return.
- When you stop indenting, the function is over.

Note that `odeint` expects the function (`diff_eq` here) to take (at least) two arguments, where the first (`y` here) is the dependent variable and the second (`x` here) is the independent variable. `odeint` needs the function to take both of those arguments (at least), even if these variables are not used in the function (for instance if they are not used in the DE).

Now ask `odeint` to generate a solution to our DE.

In [ ]:
x = linspace(-1, 5, 61)
y = odeint(diff_eq, 0, x)[:, 0]

- `linspace` creates an array of (`61`, in this case) equally-spaced elements.
- `odeint` calculates `y` for each value of `x`.
- In Python, functions are variables like any other. In this case we pass `diff_eq` as an argument to `odeint`.
- The second argument to `odeint` (`0` here) is the initial value of $y$. It must correspond to the first value of `x`.
- `odeint` returns a 2D array with 61 rows and 1 column. We need a 1D array for plotting, so we extract the first column using `[:, 0]`.

The following will plot `x` and `y` in a line plot, just like last week.

In [ ]:
data = pd.DataFrame({'x': x, 'y': y})
sns.lineplot(data=data, x='x', y='y');

Finally, to calculate $y(5)$, we realise that the values calculated by `odeint` are stored in the array `y`. So display `y`.

In [ ]:
y

Here we just want the last value. We can grab the last element of the array with `y[-1]`. (`y[-2]` gives the second last element.)

In [ ]:
y[-1]

`x[-1]` is th elast element of `x`. Check it too.

In [ ]:
x[-1]

Now we will plot multiple (approximate) solutions on the same graph. The procedure is similar, but now we need an additional `DataFrame.melt` step, to get the data into the shape that Seaborn would like it.

Technically
- `melt` is required because Seaborn likes _long_ format data, and the DataFrame we have created is in _wide_ format.
- `id_vars` says that `x` is the independent (mathematical) variable
- `value_name` says that `y` is the (common) dependent (mathematical) variable
- `var_name` is the label that will eventually appear in the plot key
- telling Seaborn to vary the hue (colour) by `initial value` results in multiple lines on the same plot

In [ ]:
x = linspace(-1, 5, 61)
data = {'x': x,
        'y(-1) = 0': odeint(diff_eq, 0, x)[:, 0],
        'y(-1) = 2': odeint(diff_eq, 2, x)[:, 0],
        'y(-1) = -2': odeint(diff_eq, -2, x)[:, 0]}
data = pd.DataFrame(data)
data = data.melt(id_vars=['x'], value_name='y', var_name='initial value')
sns.lineplot(data=data, x='x', y='y', hue='initial value');

Now let’s put the slope field and the numerical solutions together. Copy and paste the code from above where we created the quiver plot into the cell below, then copy and paste the code from above where we created the line plots below it (in the same cell).

If you have done it properly, the result should look something like this:

![](images/week-2.png)

(Changing the colour of the slopefield makes the blue solution line pop.)

## Exercises

### Slope field and DE solution plot

Plot on one figure the slopefield for the DE
\begin{align*}
\frac{\mathrm{d} y}{\mathrm{d} x} = 2.5y (1 − y),
\end{align*}
and the solutions to the initial value problems $y(0) = 0.2$, $y(0) = 0.5$ and $y(0) = 0.8$.

Start by writing down a new definition for `diff_eq` below. Do not change the function's name or inputs.

In [ ]:
def diff_eq(y, x):
    ### diff_eq implementation goes here

If you have implemented `diff_eq` correctly, the following should print "nice job".

In [ ]:
assert_almost_equal(diff_eq(0.4, 0), 0.6)
assert_almost_equal(diff_eq(0.4, 10), 0.6)
assert_almost_equal(diff_eq(1, 0), 0)
print("nice job")

Now create your graph. Note that you will have to redefine `S` (from above). You can do that using your new definition for `diff_eq` or by writing out the RHS of the equation again.

You will also have to change your definition of the meshgrid for the slopefield and the domain and initial values in the `odeint` commands. You want about 21 steps in the x and y ranges in meshgrid. 

Create the plot for the region $0 < x < 1$ and $0 < y < 1$.

### Solution at a point
What is $y(1)$ if $y(0)=0.8$?